In [1]:
import time
import math
import numpy as np
import scipy.io as sio
from numpy import linalg as la

In [2]:
#data i/o(read x_train, y_train, z_train, x_query, y_query,mu_pred)

data = sio.loadmat('whatIneed.mat')
x_train = data['x_gp']
u_train = data['u_gp']
z_train = data['z_gp']

n = len(x_train) #number of samples
d = len(x_train[0]) #dimension of the data
m = len(u_train[0]) #number of controls

u_train = u_train.reshape(n)
y_train = np.concatenate(([np.ones(n)],[u_train]), axis=0)
y_train = y_train.T      
x_train = x_train.reshape((n,d))
z_train = z_train.flatten()
x_test = x_train[0]
y_test = y_train[0]
z_test = z_train[0] #true value of the mu at query

In [3]:
#setting hyperparameters
rf_d = 100 #rf_d is dim of randomfeatures vector, to choose based on paper
mu = 0
sigma = 1
rf_mu = 0
rf_sigma = sigma * (math.pi ** 2)
sigma_n = 1 #data noise

In [4]:
#computing random features
#referring to random features as rf


fourier_transform = np.random.normal(rf_mu,rf_sigma,size =(rf_d//2,d))
samples = (sigma * math.pi)**(-rf_d/2) * fourier_transform


def rf_vec(x):
    phi = np.zeros(rf_d)
    for i,sample in enumerate(samples):
        phi[2*i] = np.sin(np.dot(sample,x))
        phi[2*i+1] = np.cos(np.dot(sample,x)) 
    phi = math.sqrt(2/rf_d) * phi
    return phi

def rf_vec_compound(x,y):
    #same sampling for different kernels?#same mu,sigma?
    phis = []
    for i in range(m+1): 
        phis = np.append(phis, y[i]*rf_vec(x))
    return phis
                         
def computeZ(x_train,y_train):
    Z = [rf_vec_compound(x_train[i],y_train[i]) for i in range(n)]
    Z = np.array(Z)
    return Z

In [5]:
#computing compound kernel
#referring to compound as c
   
def compound_kernel(x_train,y_train):
    c_kernel = np.zeros((n,n)) 
    for i in range(n):
        for j in range(n):
            c_kernel[i][j] =\
            compute_entry(x_train[i], x_train[j], y_train[i],y_train[j])
    return c_kernel

def cquery_vector(x_train, x_test, y_train, y_test):
    c_vector = np.zeros(n)
    for i in range(n):
        c_vector[i] = compute_entry(x_train[i], x_test, y_train[i], y_test)
    return c_vector
    
def compute_entry(x1,x2,y1,y2):
    ck = 0
    for i in range(m+1):
        ck += y1[i] * compute_kernel(x1,x2) * y2[i]
    return ck

def compute_kernel(x1,x2):
    return np.exp(-la.norm(x1-x2, ord=2)/sigma ** 2)
    

In [6]:
# random features training
# w = (Z'Z+simga^2I)^{-1}Z'y
rf_start = time.time()
Z = computeZ(x_train, y_train)
invZ_with_noise = la.inv(Z.T @Z + sigma_n ** 2 * np.identity((m+1)*rf_d))
rf_w = invZ_with_noise @ Z.T @ z_train


#label prediction
phi_test = rf_vec_compound(x_test,y_test)
rf_pred = np.dot(phi_test,rf_w)
rf_end = time.time()

In [7]:
# compound kernel training
# c_w = (K+simga^2I)^{-1}K(x)
ckernel_start = time.time()
c_kernel = compound_kernel(x_train,y_train)
c_vector = cquery_vector(x_train, x_test, y_train, y_test)
inv_ckernel_with_noise = la.inv(c_kernel + sigma_n**2 * np.identity(n))
c_w = inv_ckernel_with_noise @ c_vector


#label prediction
c_pred = np.dot(z_train,c_w)
ckernel_end = time.time()

In [8]:
np.set_printoptions(threshold=np.inf)
estimate = Z @ Z.T
dif = np.abs(c_kernel- estimate)
sum_dif = np.mean(dif < .01)
print(sum_dif)
for i in range(n):
    for j in range(n):
        if dif[i][j] > 1:
            print(i,j, c_kernel[i][j], estimate[i][j]);

0.021392028554431886
0 116 7.507532745575079 9.143520892276612
0 120 7.525386478656554 9.143520892276612
0 124 7.751069725253041 9.143520892276612
0 128 7.781078790941512 9.143520892276612
0 132 6.462461799935389 7.980160764808519
0 136 7.806867856306622 9.143520892276612
0 140 7.954868019697791 9.143520892276612
0 144 2.85631755183348 9.143520892276612
0 145 3.8528299552094207 9.143520892276612
0 146 1.7562433404941777 3.5932986489729353
0 147 2.301018182199834 4.343347661440464
0 148 2.839152965146432 4.88934632543223
0 149 3.85461166864286 9.143520892276612
0 150 1.456873898963351 2.9933871794523
0 151 2.003065491764114 3.8392708899284402
0 152 2.536749030395404 4.475153908263521
0 153 3.855934799275516 9.143520892276612
0 154 1.1482402687166837 2.3702378312713708
0 155 1.7041834576376087 3.319822714434961
0 156 2.2388454575457564 4.051000137808547
0 157 3.856797778154298 9.143520892276612
0 159 1.4043915869417818 2.7834168042527865
0 160 1.9460754029961262 3.6166240196807915
0 161 

In [27]:
#random features sigma computation
rfs_start = time.time()
rf_sigma = sigma**2 * phi_test.T @ invZ_with_noise @ phi_test
rfs_end = time.time()

#kernel sigma computation
ckernels_start = time.time()
kernel_sigma = compute_entry(x_test, x_test, y_test, y_test) -\
            c_vector.T @ inv_ckernel_with_noise @ c_vector
ckernels_end = time.time()

In [28]:
#print(f'{z_test=}{rf_pred=}{c_pred=}{rf_sigma=}{kernel_sigma=}') does not work for some reason
print(f'z_test={z_test} \nrf_pred={rf_pred} \nc_pred={c_pred}\n\
rf_sigma={rf_sigma} \nkernel_sigma={kernel_sigma}')
rf_mu_time = rf_end - rf_start
c_mu_time = ckernel_end - ckernel_start
print('rf_mu_time, c_mu_time', rf_mu_time, c_mu_time)

rf_sigma_time = rfs_end - rfs_start
c_sigma_time = ckernels_end - ckernels_start
print('rf_sigma_time, c_sigma_time', rf_sigma_time, c_sigma_time)

z_test=0.18107397972611733 
rf_pred=0.4121857884758133 
c_pred=0.183567986443154
rf_sigma=0.004856076008427998 
kernel_sigma=0.12267424523537152
rf_mu_time, c_mu_time 0.21096563339233398 1.1200871467590332
rf_sigma_time, c_sigma_time 0.000247955322265625 0.00037217140197753906


In [29]:
#change d    
#time constructing the matrix for socp 
#make figures?